| Class | Method |
|---|---|
|CleanDF |     replace_empty_space(df, column) |
| | replace_NaN(df, column)|
| | handle_missing_values(df)|
|InspectSource |     set_flag_analyze_all_channels()|
| | check_source_path_exists()
| | check_converted_path_exists()
| | check_format_of_json_names(list_names) |
| | get_channels_names()|
| | get_all_channels_json_names()|
| | check_missing_channels(present_channel_names) |
| | check_expected_files_exists()|
|SlackChannelsAndUsers | write_info_to_file(write_file_flag, filename, df)|
| |get_all_channels_info()|
| | get_all_users_info()|
|SlackMessages| slack_json_to_dataframe|
| | get_channel_messages_df(export_path, curr_channel_name, json_list)|
| | get_channel_users_df(channel_messages_df, users_df)|
| | add_users_info_to_messages(df_messages, df_users)|
| | ts_to_tz(original_column_name, new_column_name)|
| | extract_urls(df)|
| | user_id_to_name(df_messages, df_users|
| | parent_user_id_to_name(df_messages, df_users|
| | channel_id_to_name(df_messages, df_users)|
| | apply_excel_adjustments(file_path, curr_channel_name)|
| | get_all_messages_df()|




In [2]:
##-- Retrieve the user's input information from the file inputs.py:
import inputs

rows_to_show = inputs.rows_to_show
missing_value = inputs.missing_value
timmeshift = inputs.timmeshift
chosen_channel_name = inputs.chosen_channel_name
write_all_channels_info = inputs.write_all_channels_info
write_all_users_info  = inputs.write_all_users_info
slackexport_folder_path = inputs.slackexport_folder_path
converted_directory = inputs.converted_directory
key_wrd_text_show = inputs.key_wrd_text_show
continue_analysis = inputs.continue_analysis


In [3]:
##-- Import required packages:
import pandas as pd
from json import load
import numpy as np
from datetime import datetime
from os import listdir
from os.path import getmtime, exists, isdir, isfile
from pathlib import Path
import shutil
from urlextract import URLExtract
import re

#IP2024119   Excel's stuff
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment  
from openpyxl.utils import get_column_letter


In [4]:
class CleanDF:
    def __init__(self):
        pass
    
    def replace_empty_space(self, df, column):
        """Function to replace empty spaces "" with the string missing_value for a given column"""
        for i in range(len(df)):
            if df.at[i,column] == "":
                df.at[i,column] = missing_value  
                
    def replace_NaN(self, df, column):
        """Function to replace missing values with the string missing_value for a given column """
        df[column] = df[column].fillna(missing_value)    
        
    def handle_missing_values(self, df):
        """Function that replaces missing values in all the columns of the df"""
        df = df.replace(pd.NaT, missing_value)
        df = df.replace(np.nan, missing_value) 
        df = df.fillna(missing_value)
        return df
        

In [5]:
class InspectSource:
    def __init__(self):       
        pass
        
    #---------------------------------------------------------
    def set_flag_analyze_all_channels(self):
        if len(chosen_channel_name) < 1:
            analyze_all_channels = True 
            print('Channel(s) to analyze: All')
        else:
            analyze_all_channels = False
            print('Channel(s) to analyze: ', chosen_channel_name)
        return analyze_all_channels
    
    def check_source_path_exists(self):
        if exists(slackexport_folder_path)==False:
            print('Please enter a valid path to the source directory')
            continue_analysis = False        #  IP20241124  may be add here abort of entire code? like "sys.exit()" ?
    
    def check_converted_path_exists(self):
        converted_directory_ = f"{converted_directory}/_JSONs_converted"
        ##-- Check that     exprt_folder_path  for resulting Excels (JSONs been converted) exists:  #IP20241118
        if exists(converted_directory_)==True:
            exprt_folder_path = Path(converted_directory_)
            if exprt_folder_path.is_dir():
                print(f"The folder 'JSONs_converted' already exists in '{converted_directory_.split('JSONs')[0][:-1]}' and it will be overwritten.") #AG20241120
                shutil.rmtree(exprt_folder_path)
        Path(f"{converted_directory_}").mkdir(parents=True, exist_ok=True) #IP20241119
        return converted_directory_

    #---------------------------------------------------------
    def check_format_of_json_names(self, list_names):
        """ Iterates over all the json files in a channel's directory, and returns a list with the names of the json files 
        that have the correct format 'yyyy-mm-dd.json' """
        list_names_dates = []
        for i in range(len(list_names)):
            match = re.match(r'(\d{4})(-)(\d{2})(-)(\d{2})(.)(json)',list_names[i])
            if match!=None:
                list_names_dates.append(list_names[i])
        return list_names_dates  
    
    
    def get_channels_names(self):     # AG20241120
        """ Returns a list with the name(s) of the Slack channels to be converted.
        If analysing one channel, check that its directory exists, and default to the 0-th element of channels_names:
        channels_names = [ chosen_channel_name ] for one channel
        channels_names = [channel0, channel1, ...] for all the channels """
        if analyze_all_channels == False:
            if exists(f"{slackexport_folder_path}/{chosen_channel_name}")==False:
                self.channels_names = []
                print(f"The source directory for the channel '{chosen_channel_name}' was not found in {slackexport_folder_path}")
                continue_analysis = False
            else:
                self.channels_names = [chosen_channel_name]
        else:
            all_in_sourceDir = listdir(slackexport_folder_path)
            self.channels_names = [all_in_sourceDir[i] for i in range(len(all_in_sourceDir)) if isdir(f"{slackexport_folder_path}/{all_in_sourceDir[i]}")==True]
            
        #AG20241120: Pending to check the format of each channel's name. Having empty spaces in the name can cause problems later. 
        return self.channels_names
            
    
    def get_all_channels_json_names(self): 
        """ 
        Check the names of json files in all the channels to be converted and stores them in a list:
        all_channels_jsonFiles_dates = [ [chosen_channel_name_json0, chosen_channel_name_json1, ...] ] for one exportchannel
        all_channels_jsonFiles_dates = [ [channel0_json0, channel0_json1, ...], [channel1_json0, channel1_json1, ...], ... ] for all the channels
        """
        all_channels_jsonFiles_dates = []
        for channel in self.get_channels_names():
            channel_jsonFiles_dates = self.check_format_of_json_names( listdir(f"{slackexport_folder_path}/{channel}") )
            all_channels_jsonFiles_dates.append(channel_jsonFiles_dates)
        return all_channels_jsonFiles_dates
    
    
    def check_missing_channels(self, present_channel_names):   #AG20241127
        ##-- Get name of channels in channels.json:
        expected_channel_names = pd.read_json(f"{slackexport_folder_path}/channels.json")['name'].values
        ##-- Check that all the expected channels are in present channels:
        missing_channels = []
        for channel in expected_channel_names:
            if channel not in present_channel_names:
                missing_channels.append(channel)
        if len(missing_channels) > 0:
            return missing_channels
        else:
            return None

    
    def check_expected_files_exists(self):    
        if exists(slackexport_folder_path)==False:
            print('Please enter a valid path to the source directory')
            continue_analysis = False
        else:
            #  !!! IP2024118  need to check if exist File "channels.json"
            ##-- Check that the channels.json files exists:     # AG20241119:
            if exists(f"{slackexport_folder_path}/channels.json")==False:
                print('File "channels.json" was not found in the source directory')
                continue_analysis = False
             
            ##-- Check that the users.json files exists:
            if exists(f"{slackexport_folder_path}/users.json")==False:
                print('File "users.json" was not found in the source directory')
                continue_analysis = False
    
            ##-- Get a list with the name of the channels to be converted:
            self.channels_names = self.get_channels_names() #AG20241120: Defined routine in function  
    
            #IP20241129 it could be (maybe) useful in further with GUI, but not now)
            ##-- Check for missing channels in the source directory:       #AG20241127
            if analyze_all_channels == True:
                missing_channels = self.check_missing_channels(self.channels_names)
                if missing_channels != None:
                    print('The following channels are missing in the source directory:', missing_channels)
                    continue_analysis = True
                    #IP20241129  continue_analysis = False    ##AG: pending to prompt the user if continuing with the analysis? (Relevant for the GUI)

            ##-- Get the name of all the json files of the form "yyyy-mm-dd.json" in each channel directory:
            self.all_channels_jsonFiles_dates = self.get_all_channels_json_names() # AG20241120: Defined routine in function


In [6]:
class SlackChannelsAndUsers:
    def __init__(self):
        self.inspect_source = InspectSource()
        self.cleanDF = CleanDF()

    def write_info_to_file(self, write_file_flag, filename, df):
        if continue_analysis==False:
            print("Please review the input information")
        else:    
            if write_file_flag==True:
                slack_export_user_filename = filename        
                slack_export_user_folder_path_xlsx = f"{converted_directory}/{slack_export_user_filename}{'.xlsx'}" #_IP
                df.to_excel(slack_export_user_folder_path_xlsx, index=False) #_IP
                print(datetime.now().time(), f"Wrote file {filename}.xlsx")
                
    def get_all_channels_info(self):
        """
        This function exports the file channels.json into the dataframe all_channels_df and filters/format relevant features.
        The primary features of all_channels_df are: 
            id, name, created, creator, is_archived, is_general, members, pins, topic, purpose.
        The secondary features of 'pins' are:
            id, type, created, user, owner.
            Generally a list of dictionaries.
        The secondary features of 'topic' are:
            value, creator, last_set.
        """
        ##-- Export channels.json to dataframe    
        self.all_channels_df = pd.read_json(f"{slackexport_folder_path}/channels.json")
    
        ##-- Format relevant features on all_channels_df:
        all_json_files = []
        for i in range(len(self.all_channels_df)):
            ##-- Adds df['members']. Writes the list of members into a string separated by commnas:
            tmp_list = self.all_channels_df.at[i, 'members']
            members_str = "".join(f"{tmp_list[j]}, " for j in range(len(tmp_list)))
            self.all_channels_df.at[i,'members'] = members_str[:-2]
            ##-- Adds df['purpose']:
            self.all_channels_df.at[i,'purpose'] = self.all_channels_df.at[i,'purpose']['value']
            ##-- Adds a list with the channel's json_files with the correct format (yyyy-mm-dd.json):
            channel_path = f"{slackexport_folder_path}/{self.all_channels_df.at[i,'name']}"
            
            ##-- Check that the channel_path exists:   #IP20241118
            if exists(channel_path)==True:
                list_names_dates = self.inspect_source.check_format_of_json_names(listdir(channel_path)) #AG20241120: list_names_others not part of the output anymore
                all_json_files.append(list_names_dates)
            else:
                all_json_files.append(missing_value)  
        self.all_channels_df['json_files'] = all_json_files
        
        ##-- Keep the relevant features:
        self.all_channels_df = self.all_channels_df[['id', 'name', 'created', 'creator', 'is_archived', 'is_general', 'members', 'purpose', 'json_files']]
    
        ##-- Handle missing values or empty strings:
        for feature in ['members', 'purpose']:
            self.cleanDF.replace_empty_space(self.all_channels_df, feature)

        ##-- Write all channel's info to .xlsx files, if requested by user:
        self.write_info_to_file(write_all_channels_info, "_all_channels", self.all_channels_df)
    
    
    def get_all_users_info(self):
        """
        This function exports the file users.json into the dataframe all_users_df and filters/format relevant features.
        The primary features of all_users_df are: 
            id, team_id, name, deleted, color, real_name, tz, tz_label, tz_offset, profile, is_admin, is_owner,
            is_primary_owner, is_restricted,is_ultra_restricted, is_bot, is_app_user, updated, is_email_confirmed,
            who_can_share_contact_card, is_invited_user, is_workflow_bot, is_connector_bot.
        Among the secondary features of 'profile', there are:
            title, phone, skype, real_name, real_name_normalized, display_name, display_name_normalized, fields, 
            status_text, status_emoji, status_emoji_display_info, status_expiration, 
            avatar_hash, image_original, is_custom_image, email, huddle_state, huddle_state_expiration_ts, 
            first_name, last_name, image_24, image_32, image_48, image_72, image_192, image_512, image_1024, 
            status_text_canonical, team.
        """
        ##-- Read users.json as a dataframe:
        self.all_users_df = pd.read_json(f"{slackexport_folder_path}/users.json")
        
        ##-- Keep relevant features on all_users_df:
        for i in range(len(self.all_users_df)):
            self.all_users_df.at[i, 'display_name'] = self.all_users_df.at[i, 'profile']['display_name']
            for feature in ['title', 'real_name', 'status_text', 'status_emoji']:
                self.all_users_df.at[i, f"profile_{feature}"] = self.all_users_df.at[i, 'profile'][feature]
        self.all_users_df = self.all_users_df[['id', 'team_id', 'name', 'deleted', 'display_name', 'is_bot', 'profile_title', 'profile_real_name', 
                                     'profile_status_text', 'profile_status_emoji']]
        
        ##-- Handling missing values in all_users_df:
        for feature in ['display_name', 'name', 'team_id', 'id', 'profile_title', 'profile_real_name']:#, 'profile_status_text', 'profile_status_emoji']:
            self.cleanDF.replace_empty_space(self.all_users_df, feature) 
            
        ##-- Write all users's info to .xlsx files, if requested by user:
        self.write_info_to_file(write_all_users_info, "_all_users", self.all_users_df)

        return self.all_users_df



In [7]:
class SlackMessages:
    def __init__(self):
        self.channels_names = InspectSource().get_channels_names()
        self.all_channels_jsonFiles_dates = InspectSource().get_all_channels_json_names()                			
        self.all_users_df = SlackChannelsAndUsers().get_all_users_info()
        
    
    def slack_json_to_dataframe(self, slack_json):
        """ Function to extract channel's messages from a JSON file """
        messages_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text", 
                                            "reply_count", "reply_users_count", 
                                            "ts_latest_reply", "ts_thread", "parent_user_id"])
        for message in range(len(slack_json)):
            #if 'files' in slack_json[message] and slack_json[message]['files']:            #AG:commented out
            #    messages_df.at[message, "msg_id"] = slack_json[message]['files'][0]['id']  #AG:commented out
            if 'client_msg_id' in slack_json[message]:
                messages_df.at[message, "msg_id"] = slack_json[message]['client_msg_id']
            elif 'subtype' in slack_json[message]:                                       #AG:added
                messages_df.at[message, "msg_id"] = slack_json[message]['subtype']       #AG:added
            else:
                messages_df.at[message, "msg_id"] = missing_value #'n/a'
                
            #if 'ts' in slack_json[message]:
            #    messages_df.at[message, "ts"] = slack_json[message]['ts']
            #else:
            #    messages_df.at[message, "ts"] = missing_value  
            
            #messages_df.at[message, "user"] = slack_json[message].get('user', missing_value)  
            
            #if 'text' in slack_json[message]:
            #    messages_df.at[message, "text"] = slack_json[message]['text']
            #else:
            #    messages_df.at[message, "text"] = missing_value  
    
            
            # IP20241124 restored (otherwise missed to store timestamps)
            if 'type' in slack_json[message]:
                messages_df.at[message, "type"] = slack_json[message]['type']
            else:
                messages_df.at[message, "type"] = missing_value  
    
    
            # IP20241124 restored (otherwise missed to store timestamps)
            if 'reply_count' in slack_json[message]:
                #messages_df.at[message, "reply_count"] = slack_json[message]['reply_count']   #AG20241127: line could be deleted if using for loop at the end
                #messages_df.at[message, "reply_users_count"] = slack_json[message]['reply_users_count']  #AG20241127: line could be deleted if using for loop at the end
                messages_df.at[message, "ts_latest_reply"] = slack_json[message]['latest_reply']
            else:
                #messages_df.at[message, "reply_count"] = missing_value   #AG20241127: line could be deleted if using for loop at the end
                #messages_df.at[message, "reply_users_count"] = missing_value  #AG20241127: line could be deleted if using for loop at the end
                messages_df.at[message, "ts_latest_reply"] = missing_value   
            
            # IP20241124 restored (otherwise missed to store timestamps)
            if 'parent_user_id' in slack_json[message]:
                messages_df.at[message, "ts_thread"] = slack_json[message]['thread_ts']
                #messages_df.at[message, "parent_user_id"] = slack_json[message]['parent_user_id']  #AG20241127: line could be deleted if using for loop at the end
                messages_df.at[message, "type"] = "thread"    #IP20241124 to distinguish messages and threads
            else:
                messages_df.at[message, "ts_thread"] = missing_value 
                #messages_df.at[message, "parent_user_id"] = missing_value  #AG20241127: line could be deleted if using for loop at the end
    
            messages_df["text"] = messages_df["text"].astype(str)  #IP20241125  this fixed "FutureWarning: Setting an item of incompatible dtype is deprecated" 
    
            #IP20241125 Replace CR and LF in only the 'text' column  
            #messages_df["text"] = messages_df["text"].apply(lambda x: str(x).replace('\r\n\r\n', '\r\n `rn` ').replace('\r\r', '\r `r` ').replace('\n\n', '\n `n` ') if isinstance(x, str) else x)
            #IP20241125 Replace  CR 
            #messages_df["text"] = messages_df["text"].apply(lambda x: str(x).replace('\n', ' ') if isinstance(x, str) else x)
            #IP20241125 Replace  LF        this chosen as optimal variance
            #messages_df["text"] = messages_df["text"].apply(lambda x: str(x).replace('\r', ' ') if isinstance(x, str) else x)
                
            #AG20241122 simplified commented lines shown above to:
            features = ['ts', 'user',  'text', 'reply_count', 'reply_users_count',  'parent_user_id']  # IP20241124 :: 'type', 'ts_latest_reply', 'ts_thread' - are removed (otherwise missed to store timestamps) 
            for feature in features:
               messages_df.at[message, feature] = slack_json[message].get(feature, missing_value)    
                    
        return messages_df
        
    
    
    
    def get_channel_messages_df(self, export_path, curr_channel_name, json_list):
        """ Extracts all the messages of a given channel from all its JSON files, and stores them on a data frame """
        channel_messages_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text",
                                                    "reply_count", "reply_users_count",
                                                    "ts_latest_reply", "ts_thread", "parent_user_id"])
                                                    # ,"channel_folder", "json_name", "json_mod_date"])          #_IP
        
        ##-- Iterate over JSONs inside the current channel's folder:
        for file_day in range(len(json_list)):
            filejson_path = f"{export_path}/{curr_channel_name}/{json_list[file_day]}" #AG
            
            with open(filejson_path, encoding='utf-8') as f:
                import_file_json = load(f)
            import_file_df = self.slack_json_to_dataframe(import_file_json)
            import_file_df['json_name'] = json_list[file_day]
            import_file_df['json_mod_ts'] = getmtime(filejson_path)  #  un-ZIP of download from Ggl-Drive change ts to the non-sense :: "1980-01-01 00:00:00" 
            
            channel_messages_df = pd.concat([channel_messages_df, import_file_df], axis=0, ignore_index=True) 
        
        channel_messages_df['channel_folder'] = curr_channel_name   #IP
        return channel_messages_df
    
    
    def get_channel_users_df(self, channel_messages_df, users_df ):
        """Returns a data frame with the information of the users in current channel"""
        ##-- Initialize channel_users_df as a copy of users_df:
        channel_users_df = users_df.copy()
        ##-- Find the unique set of users in channel:
        channel_users_list = channel_messages_df['user'].unique()
        ##-- Collect the indices of the users that are NOT in the channel:
        indices_to_drop = [i for i in range(len(users_df)) if users_df.at[i,'id'] not in channel_users_list ]
        ##-- Drop the rows on indices_to_drop:
        channel_users_df.drop(channel_users_df.index[indices_to_drop], inplace=True)
        return channel_users_df

    
    def add_users_info_to_messages(self, df_messages, df_users):
        """Uses the user's id in the format U1234567789 from the df_messages to find the 
        name, display name and if the user is a bot from df_users. 
        The 'name', 'display_name' and 'is_bot' are then added as columns to df_messages"""
        for index in df_messages.index.values:
            i_df = df_users[df_users['id']==df_messages.at[index,'user']]
            if i_df['display_name'].shape[0]==0:        ##AG: 'USLACKBOT' is a special case
                df_messages.at[index, 'name'] =  df_messages.at[index, 'user']
                df_messages.at[index, 'display_name'] =  df_messages.at[index, 'user']
                df_messages.at[index, 'is_bot'] =  True
                df_messages.at[index, 'deactivated'] =  False    #IP20241121
            else:
                df_messages.at[index, 'name'] = i_df['name'].values
                df_messages.at[index, 'display_name'] = i_df['display_name'].values
                df_messages.at[index, 'is_bot'] = i_df['is_bot'].values
                df_messages.at[index, 'deactivated'] =  i_df['deleted'].values  #IP20241121
            del i_df
    
    
    def ts_to_tz(self, df, original_column_name, new_column_name):
        """Transforms timestamps in a dataframe's column to dates on the "US/Central" timezone"""
        df[original_column_name] = pd.to_numeric(df[original_column_name], errors='coerce')   #_IP
        tzs = []
        for i in range(len(df)):
            i_is_null = pd.Series(df.at[i,original_column_name]).isnull().values[0]    #AG20241120
            if i_is_null == True:
                #i_date = '0000-00-00 00:00:00'
                i_date = missing_value
            else:
                # IP20241119    #IP20241125 introduce a var "timmeshift" to adjast timezone from the 1st pfrt of code (easy tocontrol)
                i_date = pd.to_datetime(df.at[i,original_column_name], unit='s').tz_localize('UTC').tz_convert(timmeshift) #('US/Central')
                i_date = datetime.strftime(i_date,"%Y-%m-%d %H:%M:%S")
            tzs.append(i_date)
        df[[original_column_name]].astype('datetime64[s]')
        df[original_column_name] = tzs
        df.rename(columns={original_column_name: new_column_name}, inplace=True)
        
    
    def extract_urls(self, df):
        """Extracts all the url links in df['text'] and stores them as a list in df['URL']"""
        extractor = URLExtract()
        #print('len(df) = ',len(df))  #IP20241125
        for i in range(len(df)):
            urls = []
            urls = extractor.find_urls(df.at[i,'text'])
            #print('i = ', i , 'len(urls)= ', len(urls), 'urls= ', urls)  #IP20241125
            if len(urls)>0:
                urls_string = ' ;  '.join(urls)  #IP20241125  to fix  error_"ValueError: Must have equal len keys and value when setting with an iterable"
                df.at[i,'URL(s)'] = urls_string  #IP20241125 
                #print('i = ', i , 'urls= ', urls)  #IP20241125
            else:
                df.at[i,'URL(s)'] = "" # None   IP2024118
    
    
    #IP20241121 :: AG!  it should be "Add cases where the user_id is not found in users_df." >> like preserve original user_ID and added note "user_not_found"
    #IP20241121 :: AG!  in cases  user's "display_name"=="", then replace "user_ID" with "user_name"
    #IP20241121  ::  AG! :: should Add cases where the user_id is "USLACKBOT" or "SLACKBOT".
    def user_id_to_name(self, df_messages, df_users):
        """Replaces the user_id in the format <@U12345678> to the user's display_name in df_messages['text'], which happens
        when the user is mentioned in an Slack message through the option @user_name. 
         If there is no display_name, then 'user_id' is replaced with 'profile_real_name'.
         All the bots in df_users have an 'id' and 'profile_real_name' (not necessarily 'name' and 'display_id'). Their profile_real_name are:
        Zoom, Google Drive, monday.com, monday.com notifications, GitHub, Google Calendar, Loom, Simple Poll, Figma, 
        OneDrive and SharePoint, Calendly, Outlook Calendar, Rebecca Everlene Trust Company, Slack Team Emoji, New hire onboarding, 
        Welcome, Clockify - Clocking in/out, Zapier, Update Your Slack Team Icon, Jira, Google Sheets, Time Off, Trailhead, 
        Slack Team Emoji Copy, Guru, Guru, Google Calendar, Polly.
         Notice that 'USLACKBOT' and 'B043CSZ0FL7' are the only bot messages if df_messages, but they are not in df_users!
         In the replacements, the "<<>>" are used for clarity on the text, since names can generally have more than one word and many names
        can be referenced one after the other, which can lead to confusion when reading.
        """
        for i in range(len(df_messages)):
            text = df_messages.at[i,'text']
            matches = re.findall(r'<+@[A-Za-z0-9]+>',text)
            if len(matches)>0:
                for match in matches:
                    user = match[2:-1]
                    # AG20241122: begin
                    if user in df_users['id'].values:
                        name = df_users[df_users['id']==user]['display_name'].values[0]
                        is_bot = df_users[df_users['id']==user]['is_bot'].values[0]   
                        if is_bot==True:
                            name = df_users[df_users['id']==user]['profile_real_name'].values[0] + ' (bot)'
                        elif name == missing_value:
                            name = df_users[df_users['id']==user]['profile_real_name'].values[0]
                    else: 
                        name = f"{user} (user not found)"  ## Case for USLACKBOT and B043CSZ0FL7, since they are technically not in df_users!
                    # AG20241122: end
                    text = re.sub(f"<@{user}>", f"@{name}@", text)  #AG20241122: Added "<>" (see function's documentation) 
                    
                    #IP20241121: AG :: should Add cases where the user_id is not found in users_df.
                    #IP20241124:  issue above not solved
    
                    #IP20241121: AG :: should Add cases where the user_id is "USLACKBOT" or "SLACKBOT".
                    #IP20241124:  issue above not solved (or explane - how solved, if solved). Show cell with examples 
                df_messages.at[i,'text'] = text
    
    
    #AG20241122: defined routine that was inside user_id_to_name_test to its own function:
    def parent_user_id_to_name(self, df_messages, df_users):
        # IP20241121   "parent_user_id"  substitution
        '''Replaces the user_id in the format "UA5748HE" to the user's display_name in df_messages['parent_user_id']'''
        for i in range(len(df_messages)):
            #text1 = df_messages.at[i,'parent_user_id']
            #matches = re.findall(r'\bU[A-Za-z0-9]+\b',text1)
            #if len(matches)>0:
            #    for match in matches:
            #        user1 = match   
                    #print("i= ", i, "user1=", user1)   #IP20241121:
            #        if user1 == "SLACKBOT" or user1 == "USLACKBOT":
            #            continue
            #        name1 = df_users[df_users['id']==user1]['display_name'].values[0]
            #        text1 = re.sub(f"{user1}", f"{name1}", text1)
                    #IP20241121: should Add cases where the user_id is not found in users_df.
            #    df_messages.at[i,'parent_user_id'] = text1
    
            #AG20241122: Propose simplifying a bit (since 'matches' will always have the one element in df_messages['parent_user_id'])
            user = df_messages.at[i,'parent_user_id']
            if user!=missing_value:
                name = df_users[df_users['id']==user]['display_name'].values
                if user in df_users['id'].values:
                    is_bot = df_users[df_users['id']==user]['is_bot'].values
                    if is_bot==True:
                        name = df_users[df_users['id']==user]['profile_real_name'].values + ' (bot)'
                    elif name == missing_value:
                        name = df_users[df_users['id']==user]['profile_real_name'].values
                else:
                    name = user+' (user not found)'
                df_messages.at[i,'parent_user_id'] = name
            
    
    def channel_id_to_name(self, df_messages, df_users):
        """Replaces <#channel_id|channel_name> to channel_name in df_messages['text'], which happens
        when the channel is mentioned in an Slack message through the option #channel_name"""
        for i in range(len(df_messages)):
            text = df_messages.at[i,'text']
            matches = re.findall(r'#+[A-Za-z0-9]+\|',text)
            if len(matches)>0:
                for match in matches:
                    text = re.sub(match, "", text)
                    text = re.sub(r"<+\|", "<", text)
                df_messages.at[i,'text'] = text

    
    
    def apply_excel_adjustments(self, file_path, curr_channel_name):
        """ Excel file formatting/adjustments with  openpyxl (IP) """
        wb = load_workbook(file_path)
        ws = wb.active
        #
        
        #AG: ---------------------------------------------------
        #AG: Suggest to move to function set_cells_width. (Although lines below get overwritten later)
        ##-- Set the column width
        column_widths = {
            'A': 12, 'B': 19, 'C': 15,'D': 8, 'E': 35, 'F': 5, 'G': 5, 'H': 17, 'I': 17, 'J': 15, 
            'K': 19, 'L': 19, 'M': 19, 'N': 13, 'O': 25 , 'P': 7 , 'Q': 6  , 'R': 37      
        }
        ###-- Apply the column widths
        for col, width in column_widths.items():
            ws.column_dimensions[col].width = width
        

        
        #AG: ---------------------------------------------------
        ##AG: Suggest to move to the function set_font_color
        ##--  Apply font color to all cells in column 
        font_color = "0707C5"  
        for cell in ws['E']:   #text
            cell.font = Font(color=font_color)
        font_color = "c10105"  
        for cell in ws['J']:   #parent_user_id
            cell.font = Font(color=font_color)
        

        
        #AG: ---------------------------------------------------
        ##AG: Suggest to collect in function set_cell_allignment
        ##-- Loop through each cell in the column_"E" >> 'text'  and replace CR+LF    #IP20241125
        #    also, set alignments
        for row in ws.iter_rows(min_col=5, max_col=5, min_row=2, max_row=ws.max_row):
            for cell in row:
                if isinstance(cell.value, str):  # Check if the cell contains text
                    # Replace CR (carriage return) and LF (line feed) with a space
                    cell.value = cell.value.replace('\r\n', ' ').replace('\r', ' ').replace('\n\n', '\n')
                    cell.alignment = Alignment(wrap_text=False, vertical="top", horizontal="left")
        
        
    
        #AG: ---------------------------------------------------
        #AG: Suggest to move this reordering to the funtion get_all_messages. Much more direct to do it with dataframes in 1 line of code
        #IP20241120  re-order columns  
        #   
        # Specify the column to move  
        col_to_move_indx = 13    # N-of-clmn==(index)+1
        col_to_insert_indx = 4
        columns = list(ws.columns) # Get all columns
        col_to_move = columns[col_to_move_indx]
        col_to_insert = columns[col_to_insert_indx]  
        col_data = [cell.value for cell in col_to_move] # Get the data in the column to move
        ws.delete_cols(col_to_move_indx+1)  # Remove the column from its current position 
        ws.insert_cols(col_to_insert_indx)  # Insert the column at the destination position
        for row_idx, value in enumerate(col_data, start=1):
            ws.cell(row=row_idx, column=col_to_insert_indx, value=value)
    
        col_to_move_indx = 14    # N-of-clmn==(index)+1
        col_to_insert_indx = 5
        # Get all columns
        columns = list(ws.columns)
        col_to_move = columns[col_to_move_indx]
        col_to_insert = columns[col_to_insert_indx]  
        # Get the data in the column to move
        col_data = [cell.value for cell in col_to_move]
        # Remove the column from its current position 
        ws.delete_cols(col_to_move_indx+1)
        # Insert the column at the destination position
        ws.insert_cols(col_to_insert_indx)   
        for row_idx, value in enumerate(col_data, start=1):
            ws.cell(row=row_idx, column=col_to_insert_indx, value=value)
        

        
        #AG: ---------------------------------------------------
        ##AG: Suggest to move to the function set_cells_width. (Although lines below get overwritten later)
        ##AG column_widths_per_name_to_use_in_function = {
        ##AG    'msg_id': 12, 'msg_date': 19, 'user': 15,'name': 19, 'display_name': 19, 'type': 8, 'text': 35, 'reply_count': 5, 'reply_users_count': 5, 'latest_reply_date': 17, 
        ##AG    'thread_Date': 17, 'parent_user_id': 15, 'json_name': 19, 'json_mod_date': 19, 'channel_folder': 25 , 'is_bot': 7 , 'deactivated': 6, 'URL(s)': 37    
        ##AG} 
        ##-- re-Set the column width AFTER moving columns  IP20241124 (preserve in code, if further column-moving will be changed)
        column_widths = {
            'A': 12, 'B': 19, 'C': 15,'D': 19, 'E': 19, 'F': 8, 'G': 35, 'H': 5, 'I': 5, 'J': 17, 
            'K': 17, 'L': 15, 'M': 19, 'N': 19, 'O': 25 , 'P': 7 , 'Q': 6, 'R': 37    
        }
        ##-- Apply the column widths
        for col, width in column_widths.items():
            ws.column_dimensions[col].width = width
        
        
        
        #AG: ---------------------------------------------------
        #AG: Suggest to move this reordering to the funtion get_all_messages. Much more direct to do it with dataframes in 1 line of code
        # IP20241124 move "deactivated" column
        col_to_move_indx = 16    # N-of-clmn==(index)+1
        col_to_insert_indx = 6
        # Get all columns
        columns = list(ws.columns)
        col_to_move = columns[col_to_move_indx]
        col_to_insert = columns[col_to_insert_indx]  
        # Get the data in the column to move
        col_data = [cell.value for cell in col_to_move]
        # Remove the column from its current position 
        ws.delete_cols(col_to_move_indx+1)
        # Insert the column at the destination position
        ws.insert_cols(col_to_insert_indx)   
        for row_idx, value in enumerate(col_data, start=1):
            ws.cell(row=row_idx, column=col_to_insert_indx, value=value)
        
        # IP20241124 move "is_bot" column
        col_to_move_indx = 16    # N-of-clmn==(index)+1
        col_to_insert_indx = 7
        # Get all columns
        columns = list(ws.columns)
        col_to_move = columns[col_to_move_indx]
        col_to_insert = columns[col_to_insert_indx]  
        # Get the data in the column to move
        col_data = [cell.value for cell in col_to_move]
        # Remove the column from its current position 
        ws.delete_cols(col_to_move_indx+1)
        # Insert the column at the destination position
        ws.insert_cols(col_to_insert_indx)   
        for row_idx, value in enumerate(col_data, start=1):
            ws.cell(row=row_idx, column=col_to_insert_indx, value=value)

        
        #AG: ---------------------------------------------------
        #AG: Suggest to move to function set_cell_allignment
        ##-- Data align-to-left  IP20241124  (excluding 1st row)
        for row in ws.iter_rows(min_col=10, max_col=11, min_row=2, max_row=ws.max_row):
            for cell in row:
                cell.alignment = Alignment(horizontal='center')   # 'left'
                if isinstance(cell.value, (int, float)):
                    cell.font = Font(size=12, bold=True)
                    
     
        #AG: ---------------------------------------------------
        #AG: Suggest to move to function set_format_first_row:
        #  first row (Row 1) formattings
        ##-- Freeze the first row (Row 1)
        ws.freeze_panes = 'A2'
        ##-- Set font size and bold for the first row
        font = Font(size=9, bold=True)
        ##-- Set the height of the first row
        ws.row_dimensions[1].height = 43 
        ##-- Define the RGB color
        fill = PatternFill(start_color="e7c9fb", end_color="e7c9fb", fill_type="solid")
        ##-- Apply the color, font formatting to the 1st row (Header row)
        for cell in ws[1]:
            cell.font = font
            cell.fill = fill
            #cell.alignment = Alignment(wrap_text=True) # Set wrap text for the cells in the first row 
            cell.alignment = Alignment(wrap_text=True, vertical="top", horizontal="left")
        

        #AG: ---------------------------------------------------
        #AG: Suggest to move to function set_font_color
        font_color = "c10105"  #IP font_color User_name
        for cell in ws['E']: 
            cell.font = Font(color=font_color)
        

        #AG: ---------------------------------------------------
        #AG: Suggest to move to function set_cell_color
        #IP20241121  fill_color when  -> "is_bot"==True  -> message's "type"=="thread"
        fill_bot = PatternFill(start_color="FBBF8F", end_color="FBBF8F", fill_type="solid")
        fill_thread = PatternFill(start_color="FBFB99", end_color="FBFB99", fill_type="solid")
        last_row = ws.max_row
        for i in range(2, last_row + 1):
            if ws[f'g{i}'].value == "True" or ws[f'g{i}'].value == True:
                for col in ['C', 'D', 'E', 'F', 'G']:
                    ws[f'{col}{i}'].fill = fill_bot
            if ws[f'H{i}'].value == "thread":
                for col in ['H', 'I']:
                    ws[f'{col}{i}'].fill = fill_thread
        
        
        #AG: ---------------------------------------------------
        #IP20241129     "weekly_report" separation
        ##--    IP20241203  set widths for "weekly-report" columns
        for col_num in range(19, 33):
            col_letter = get_column_letter(col_num)   
            ws.column_dimensions[col_letter].width = 25
        
        ##-- weekly-report keywords setting:
        keywrds_wkly_report = ["Weekly Report:","Project Name:","Working on:", "Progress and Roadblocks:", "Progress:", "Roadblocks:", 
                                        "Plans for the following week:", "Meetings:"]  
        ##-- fill for Weekly-Report's titles
        fill_wkrep_titles = PatternFill(start_color="CDB5B7", end_color="CDB5B7", fill_type="solid")
    
        ##-- Get the index/letter of the last used column 
        lst_col_index = ws.max_column              # IP20241215  !!!           refactor to last filled "title-of-column-in-1st-row"
        lst_col_lttr = get_column_letter(lst_col_index)
        ##-- set columns-titles according to the  keywrds_wkly_report
    
        wkrep_tech_title = "weekly-rep-all"  # column for tech purpose - start-&-end positions of keywords 
        ws.cell(row=1, column=lst_col_index+1).value = wkrep_tech_title
        ws.cell(row=1, column=lst_col_index+1).fill = fill_wkrep_titles
        #
        for col_idx, value in zip(range(lst_col_index + 2, lst_col_index + len(keywrds_wkly_report) + 2), keywrds_wkly_report):
            cell = ws.cell(row=1, column=col_idx)
            cell.value = value
        
            col_letter = get_column_letter(col_idx)  # Convert column number to letter
            ws.column_dimensions[col_letter].width = 25
            ws[f'{col_letter}1'].fill = fill_wkrep_titles
    
        ##-- find column for "text" of messages
        text_to_find = 'text'
        for cell in ws[1]:  # Sheet row 1 is accessed using sheet[1]
            if text_to_find.lower() in str(cell.value).lower():  # Case-insensitive search
                clmn_lttr_text = get_column_letter(cell.column)  # Get the column letter
        #print('clmn_letter_text= ',clmn_lttr_text)
    
        #
        ##-- find keywords positions in cells of the column for "text" 
        for i in range(2, last_row + 1):
            key_wrds_text = []
            cell_value = ws[f'{clmn_lttr_text}{i}'].value
            cell_value = str(cell_value).replace("*", "")     #IP20241215  replacement all asterrisks in the "text"  
            if isinstance(cell_value, str):
                for keyword in keywrds_wkly_report:
                    position = cell_value.lower().find(keyword.lower())   # .lower()
                    if position != -1:
                        key_wrds_text.append((i, keyword, position, position + len(keyword)))
                        key_wrds_text_sorted = sorted(key_wrds_text, key = lambda x: x[2])
            if key_wrds_text:
                print ( "i= ", i, " len(key_wrds_text_sorted)= ", len(key_wrds_text_sorted))
                ##-- IP20241215 delete dumb keywords, which are just a part of some complex keyword 
                if len(key_wrds_text_sorted) > 1 :
                    for j in range(len(key_wrds_text_sorted)-1 , 0, -1):   
                        print ( "j= ", j,  key_wrds_text_sorted[j][1])
                        if key_wrds_text_sorted[j][2]  >= key_wrds_text_sorted[j-1][2] and key_wrds_text_sorted[j][2]  < key_wrds_text_sorted[j-1][3]:
                            print ( "i= ", i, "key_wrds to delete= ", key_wrds_text_sorted[j][1] )
                            del key_wrds_text_sorted[j]
                #
                key_wrds_text_sorted_str = '; '.join([f"'{match[1]}' at {match[2]}-{match[3]}" for match in key_wrds_text_sorted])
                #
                ws.cell(row=i, column=lst_col_index+1, value=key_wrds_text_sorted_str)
                print("i= ", i, "key_wrds_txt_srtd= ", key_wrds_text_sorted)
                #print("i= ", i, "key_wrds_text_sorted_str= ", key_wrds_text_sorted_str)
                #
                for j in range(0, len(key_wrds_text_sorted)):    #item in key_wrds_text_sorted:
                    for cell_1 in ws[1]:   
                        title_item = key_wrds_text_sorted[j][1]
                        if title_item.lower() in str(cell_1.value).lower():
                            item_clmn=cell_1.column
                    #for j in range(len(key_wrds_text_sorted) - 1):   
                    
                    if j + 1  < len(key_wrds_text_sorted):
                        next_item = key_wrds_text_sorted[j + 1]
                        end_position = next_item[2]
                    else: 
                        next_item = ("", "", "", "")
                        end_position = len(str(cell_value))
                    print("i= ", i, "len(key_wrds_txt_srt)= ", len(key_wrds_text_sorted), "item_j= ", j, " item_clmn= ", item_clmn, "next_item[2]= ", next_item[2], f" '{key_wrds_text_sorted[j][1]}'; {key_wrds_text_sorted[j][3]}; {end_position}")
                    #
                    if key_wrd_text_show != True:    # print or not keyword as a prefix) in tthe cell 
                        key_wrd_text = ''
                    else:
                        key_wrd_text = f"'{key_wrds_text_sorted[j][1]}' "
                    #
                    cell_keywrd_value = f"{key_wrd_text}{cell_value[key_wrds_text_sorted[j][3]: end_position]}"    #IP20241215
                    #IP20241215  Replace CR (carriage return) and LF (line feed) with a space
                    cell_keywrd_value = cell_keywrd_value.replace('\r\n', ' ').replace('\r', ' ').replace('\n\n', '\n')
                    ws.cell(row=i, column=item_clmn, value = cell_keywrd_value)            #IP20241215
                    #ws.cell(row=i, column=item_clmn, value=f"{key_wrd_text}{cell_value[key_wrds_text_sorted[j][3]: end_position]}")           
        
        
        
        #AG: ---------------------------------------------------
        #AG: Suggest to move this reordering to the funtion get_all_messages. Much more direct to do it with dataframes in 1 line of code
        ##-- Delete columns  ::   json_name 	json_mod_date	channel_folder    #IP20241125
        #    this columns are tech only, for development and debug, not for PMs
        clmns_to_delete = ["json_name",	"json_mod_date", "channel_folder", wkrep_tech_title]
        for clmn_to_delete in clmns_to_delete:
            for cell in ws[1]:  
                if clmn_to_delete.lower() in str(cell.value).lower():  
                    ws.delete_cols(cell.column) 
                    break

        
        #AG: ---------------------------------------------------
        #AG: Suggest to move to function set_cell_width
        ##-- re-Set the column width  IP20241205
        for col in range(15,36):
            clmn_lttr = get_column_letter(col)
            ws.column_dimensions[clmn_lttr].width = 25 
        column_widths = {
            'msg_id': 12, 'msg_date': 19, 'user': 15,'name': 19, 'display_name': 19, 'deactivated': 7, 'is_bot': 7, 
            'type':8, 'text':35, 'reply_count': 5, 'reply_users_count': 5, 'latest_reply_date': 17, 'thread_date': 17, 'parent_user_id': 25, 'URL(s)': 37      
        }
        for col, width in column_widths.items():
            for cell in ws[1]:  
                if col.lower() == str(cell.value).lower():  
                    clmn_lttr = get_column_letter(cell.column)
                    ws.column_dimensions[clmn_lttr].width = width 
                    break
        
        #AG: ---------------------------------------------------
        #AG: Suggest to move to function set_cell_allignment
        ##-- Data align-to-top  IP20241215  (excluding 1st row)
        for row in ws.iter_rows(min_col=1, max_col=35, min_row=2, max_row=ws.max_row):
            for cell in row:
                cell.alignment = Alignment(vertical='top')   
        
        #AG: ---------------------------------------------------
        #AG: Suggest to move to function rename_sheet
        ##-- Rename the sheet
        ws_title = curr_channel_name 
        ws_title = ws_title[:31]
        ws.title = ws_title 

        #AG: ---------------------------------------------------
        #AG: Suggest to move to function save_changes
        ##-- Save the changes to the Excel file
        wb.save(file_path)
        

        
    def get_all_messages_df(self):
        if continue_analysis==False:
            print("Please review the input information")
        else:    
            ##-- Iterate over channel's folders:
            dfs_list = []
            print(datetime.now().time(), 'Starting loop over channels', '\n')
            for i_channel in range(len(self.channels_names)):
        
                ##-- Define the name of the current channel and the source path containing its json files:
                curr_channel_name = self.channels_names[i_channel] 
                parentfolder_path = f"{slackexport_folder_path}/{curr_channel_name}" 
                print(curr_channel_name, datetime.now().time(), ' Set-up channel name and path to directory')
                
                ##-- Collect all the current_channel's messages in channel_messages_df through the function get_channel_messages_df:
                json_list = self.all_channels_jsonFiles_dates[i_channel]
                channel_messages_df = self.get_channel_messages_df(slackexport_folder_path, curr_channel_name, json_list)  
                print(curr_channel_name, datetime.now().time(), ' Collected channel messages from the json files')
                #
                #IP20241121 move to separate folders-without-messages-JSONs
                if len(channel_messages_df)<1:
                    print("for the folder ",curr_channel_name,"messages_number= ",len(channel_messages_df),"there is no channel's folder", '\n')
                    continue    
        
                ##-- Collect all the users in the current channel through the function get_channel_users_df:
                channel_users_df = self.get_channel_users_df(channel_messages_df, self.all_users_df )
                print(curr_channel_name, datetime.now().time(), ' Collected users in current channel')
                
                ##-- Use channel_users_df to fill-in the user's information in channel_messages_df: 
                self.add_users_info_to_messages(channel_messages_df, channel_users_df)
                print(curr_channel_name, datetime.now().time(), ' Included the users information on channel_messages_df')
                
                ##-- Replace user and team identifiers with their display_names whenever present in a message:
                #user_id_to_name(channel_messages_df, users_df) 
                self.user_id_to_name(channel_messages_df, channel_users_df) 
                self.channel_id_to_name(channel_messages_df, channel_users_df)
                self.parent_user_id_to_name(channel_messages_df, channel_users_df) #AG20241122: routine defined in its own function
                print(curr_channel_name, datetime.now().time(), " User's id replaced by their names in messages")
        
                ##-- Extract hyperlinks from messages, if present (extracted as a list; edit if needed):
                self.extract_urls(channel_messages_df)
                print(curr_channel_name, datetime.now().time(), ' URLs extracted from messages')
        
                ##-- Change format of the time in seconds to a date in the CST time-zone: (Pending 'ts_latest_reply' and 'ts_thread'!)
                #channel_messages_mindate = pd.to_datetime(np.float64(channel_messages_df['ts']), unit='s').min().date()   #AG20241120: Can be deleted
                #channel_messages_maxdate = pd.to_datetime(np.float64(channel_messages_df['ts']), unit='s').max().date()   #AG20241120: Can be deleted
                self.ts_to_tz(channel_messages_df, 'ts', 'msg_date')
                self.ts_to_tz(channel_messages_df, 'json_mod_ts', 'json_mod_date')
                self.ts_to_tz(channel_messages_df, 'ts_latest_reply', 'latest_reply_date')
                self.ts_to_tz(channel_messages_df, 'ts_thread', 'thread_date')
                print('main_analysys ->>',curr_channel_name, "  ", datetime.now().time(), ' Formated the dates and times in the dataframe')
                    
                ##-- Reorder the columns in channel_messages_df:
                #columns_order = ['msg_id', 'msg_date', 'user', 'name', 'display_name', 'deactivated', 'is_bot', 'type', 'text', 'reply_count', 'reply_users_count', 'latest_reply_date', 'thread_date', 'parent_user_id', 'URL(s)']
                #channel_messages_df = channel_messages_df[columns_order]
                
                ##-- Sort the dataframe by msg_date:
                channel_messages_df.sort_values(by='msg_date', inplace=True, ignore_index=True)
                
                ##-- Write channel_messages_df to a .xlsx file:
                channel_messages_mindate = channel_messages_df['msg_date'].min().split(" ")[0]
                channel_messages_maxdate = channel_messages_df['msg_date'].max().split(" ")[0]
                channel_messages_maxdate = channel_messages_df['msg_date'].max().split(" ")[0]
                channel_messages_filename = f"{curr_channel_name}_{channel_messages_mindate}_to_{channel_messages_maxdate}"
                channel_messages_folder_path = f"{converted_directory}/{channel_messages_filename}.xlsx"
                channel_messages_df.to_excel(f"{channel_messages_folder_path}", index=False)
                self.apply_excel_adjustments(f"{channel_messages_folder_path}",curr_channel_name)  #AG: defined this routine in the function apply_excel_adjustments
                print(curr_channel_name, datetime.now().time(), ' Wrote curated messages to xlsx files', '\n')
        
                dfs_list.append(channel_messages_df)
                
        print(datetime.now().time(), 'Done')
        
        return channel_messages_df
        

In [8]:
##-- Initialize constructor of the class InspectSource:
inspect_source = InspectSource()
##-- Check validity of input paths:
analyze_all_channels = inspect_source.set_flag_analyze_all_channels()
inspect_source.check_source_path_exists()
converted_directory = inspect_source.check_converted_path_exists()
##-- Check for expected files:
inspect_source.check_expected_files_exists()
##-- Retrieve variables:
channels_names = inspect_source.channels_names
all_channels_jsonFiles_dates = inspect_source.all_channels_jsonFiles_dates


##-- Initialize constructor of the class SlackChannelAndUsers:
scu = SlackChannelsAndUsers()
##-- Execute the main functions of the class:
scu.get_all_channels_info()
scu.get_all_users_info()
##-- Retrieve variables:
all_channels_df = scu.all_channels_df
all_users_df = scu.all_users_df


##-- Initialize constructor of the class SlackMessages:
sm = SlackMessages()
##-- Execute the main function of the class:
channel_messages_df = sm.get_all_messages_df()

channel_messages_df.head()

Channel(s) to analyze:  think-biver-sunday-checkins
The folder 'JSONs_converted' already exists in '/home/agds/Downloads/' and it will be overwritten.
11:02:58.059716 Wrote file _all_channels.xlsx
11:02:58.529926 Wrote file _all_users.xlsx
11:02:58.941506 Wrote file _all_users.xlsx
11:02:58.941639 Starting loop over channels 

think-biver-sunday-checkins 11:02:58.941653  Set-up channel name and path to directory
think-biver-sunday-checkins 11:02:59.467895  Collected channel messages from the json files
think-biver-sunday-checkins 11:02:59.485353  Collected users in current channel
think-biver-sunday-checkins 11:02:59.803324  Included the users information on channel_messages_df
think-biver-sunday-checkins 11:02:59.894049  User's id replaced by their names in messages
think-biver-sunday-checkins 11:03:02.215594  URLs extracted from messages
main_analysys ->> think-biver-sunday-checkins    11:03:02.837074  Formated the dates and times in the dataframe
i=  5  len(key_wrds_text_sorted)=  1

,msg_id,msg_date,user,type,text,reply_count,reply_users_count,latest_reply_date,thread_date,parent_user_id,json_name,json_mod_date,channel_folder,name,display_name,is_bot,deactivated,URL(s)
0,da57854a-a32d-454c-9c4b-0a0778167812,2024-11-01 12:16:28,U07KJ7SBYRJ,message,"Hi Team,\n\nUpdate on Scapegoat Writing Servic...",n/d,n/d,n/d,n/d,n/d,2024-11-01.json,2024-12-01 11:50:42,think-biver-sunday-checkins,shehafaakil97,Mohamed Shehaf Aakil Sharfudeen,False,False,Monday.com ; Monday.com
1,6da027f9-6deb-457c-9a80-e97a7669db5b,2024-11-01 13:24:32,U05CKAKNYAV,message,"Hi everyone, I updated some edits to charts an...",n/d,n/d,n/d,n/d,n/d,2024-11-01.json,2024-12-01 11:50:42,think-biver-sunday-checkins,samuelnusbaum01,Sam Nusbaum,False,False,
2,c24e5ef0-8788-4025-830d-7d9e6ca8d59f,2024-11-01 13:27:23,U06NRKTHR34,message,"Hi Team,\n• Working on data entry for grocery ...",n/d,n/d,n/d,n/d,n/d,2024-11-01.json,2024-12-01 11:50:42,think-biver-sunday-checkins,gargakshat6,Akshat Garg,False,False,
3,89C99856-A17D-4BC2-991B-B34DE7CD4F11,2024-11-01 13:31:59,U07N8SLKARF,message,"Hey Everyone, Apologies for the delay \nHere a...",n/d,n/d,n/d,n/d,n/d,2024-11-01.json,2024-12-01 11:50:42,think-biver-sunday-checkins,007aaronjohn,Aaron John,False,False,
4,3fb1d0e3-8581-465b-8a08-2e5c8fd86b8b,2024-11-01 13:57:17,U07KBQG5S2K,message,"Hi Team,\n\n• _What am I working on_:- Creatin...",n/d,n/d,n/d,n/d,n/d,2024-11-01.json,2024-12-01 11:50:42,think-biver-sunday-checkins,patil.vira,Viraj,False,False,


In [35]:
##-- Class with suggested changes in the function apply_excel_adjustments:
class ExcelFormat():
    def __init__(self, file_path, curr_channel_name):
        self.file_path = file_path
        self.curr_channel_name = curr_channel_name
        self.wb = load_workbook(self.file_path)
        self.ws = self.wb.active

    def set_cell_width(self):
        column_widths = {
            'msg_id': 12, 'msg_date': 19, 'user': 15,'name': 19, 'display_name': 19, 'deactivated': 7, 'is_bot': 7, 
            'type':8, 'text':35, 'reply_count': 5, 'reply_users_count': 5, 'latest_reply_date': 17, 'thread_date': 17, 'parent_user_id': 25, 'URL(s)': 37      
        }
        for col, width in column_widths.items():
            for cell in self.ws[1]:  
                if col.lower() == str(cell.value).lower():  
                    clmn_lttr = get_column_letter(cell.column)
                    self.ws.column_dimensions[clmn_lttr].width = width 
                    break

    def set_cell_color(self):
        #IP20241121  fill_color when  -> "is_bot"==True  -> message's "type"=="thread"
        fill_bot = PatternFill(start_color="FBBF8F", end_color="FBBF8F", fill_type="solid")
        fill_thread = PatternFill(start_color="FBFB99", end_color="FBFB99", fill_type="solid")
        last_row = self.ws.max_row
        for i in range(2, last_row + 1):
            if self.ws[f'g{i}'].value == "True" or self.ws[f'g{i}'].value == True:
                for col in ['C', 'D', 'E', 'F', 'G']:
                    self.ws[f'{col}{i}'].fill = fill_bot
            if self.ws[f'H{i}'].value == "thread":
                for col in ['H', 'I']:
                    self.ws[f'{col}{i}'].fill = fill_thread
    
    def set_font_color(self):
        ##-- Text:
        for cell in self.ws['I']:    #AG:changed from E to I
            cell.font = Font(color="0707C5")
        ##-- Parent_user_id:
        for cell in self.ws['N']:    #AG:changed from J to N
            cell.font = Font(color="c10105")
        ##-- Display_name:
        for cell in self.ws['E']:  
            cell.font = Font(color="c10105")

    
    def set_cell_allignment(self):
        ##-- Loop through each cell in the column_"E" >> 'text'  and replace CR+LF    #IP20241125
        #    also, set alignments
        for row in self.ws.iter_rows(min_col=9, max_col=9, min_row=2, max_row=self.ws.max_row):   #AG: Changed from 5 to 9
            for cell in row:
                if isinstance(cell.value, str):  # Check if the cell contains text
                    # Replace CR (carriage return) and LF (line feed) with a space
                    cell.value = cell.value.replace('\r\n', ' ').replace('\r', ' ').replace('\n\n', '\n')
                    cell.alignment = Alignment(wrap_text=False, vertical="top", horizontal="left")

        ##-- Data align-to-left  IP20241124  (excluding 1st row)
        for row in self.ws.iter_rows(min_col=10, max_col=11, min_row=2, max_row=self.ws.max_row):
            for cell in row:
                cell.alignment = Alignment(horizontal='center')   # 'left'
                if isinstance(cell.value, (int, float)):
                    cell.font = Font(size=12, bold=True)

        ##-- Data align-to-top  IP20241215  (excluding 1st row)
        for row in self.ws.iter_rows(min_col=1, max_col=35, min_row=2, max_row=self.ws.max_row):
            for cell in row:
                cell.alignment = Alignment(vertical='top')   

    
    def set_format_first_row(self):
        ##-- Freeze the first row (Row 1)
        self.ws.freeze_panes = 'A2'
        ##-- Set font size and bold for the first row
        font = Font(size=9, bold=True)
        ##-- Set the height of the first row
        self.ws.row_dimensions[1].height = 43 
        ##-- Define the RGB color
        fill = PatternFill(start_color="e7c9fb", end_color="e7c9fb", fill_type="solid")
        ##-- Apply the color, font formatting to the 1st row (Header row)
        for cell in self.ws[1]:
            cell.font = font
            cell.fill = fill
            #cell.alignment = Alignment(wrap_text=True) # Set wrap text for the cells in the first row 
            cell.alignment = Alignment(wrap_text=True, vertical="top", horizontal="left")

    
    def rename_sheet(self):
        ws_title = self.curr_channel_name 
        ws_title = ws_title[:31]
        self.ws.title = ws_title 

    def save_changes(self):
        self.wb.save(self.file_path)
        


##-- Select and reorder the dataframe columns:
columns_order = ['msg_id', 'msg_date', 'user', 'name', 'display_name', 'deactivated', 'is_bot', 'type', 'text', 'reply_count', 'reply_users_count', 'latest_reply_date', 'thread_date', 'parent_user_id', 'URL(s)']
df_TEST = channel_messages_df[columns_order]

##-- Save dataframe to Excel file:
channel_messages_mindate = df_TEST['msg_date'].min().split(" ")[0]
channel_messages_maxdate = df_TEST['msg_date'].max().split(" ")[0]
channel_messages_filename = f"{chosen_channel_name}_{channel_messages_mindate}_to_{channel_messages_maxdate}_TEST"
channel_messages_folder_path = f"{converted_directory}/{channel_messages_filename}.xlsx"
df_TEST.to_excel(f"{channel_messages_folder_path}", index=False)

##-- Format Excel file (Parsing of check-in messages in apply_excel_adjustments not done here):
ef = ExcelFormat(f"{channel_messages_folder_path}",chosen_channel_name)
ef.set_cell_width()
ef.set_cell_color()
ef.set_font_color()
ef.set_cell_allignment()
ef.set_format_first_row()
ef.rename_sheet()
ef.save_changes()
